<a href="https://colab.research.google.com/github/takuyats/ngboost/blob/master/%E6%96%87%E6%9B%B8%E5%88%86%E9%A1%9E%E3%81%A7%E8%87%AA%E7%84%B6%E8%A8%80%E8%AA%9E%E5%87%A6%E7%90%86%E3%81%AB%E8%A7%A6%E3%82%8C%E3%82%8B_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 文書分類で自然言語処理に触れる
<提供>
- 高橋寛治 [@kanji250tr](https://twitter.com/kanji250tr)
- [スライド資料リンク](https://docs.google.com/presentation/d/e/2PACX-1vQHjfOTrH-xu49F6iINswP99AedOSgkYLlc4smMkkCi-GJ1WrA3CKAPr8epiRR8Cy4t021tP_6MWf7f/pub?start=false&loop=false&delayms=3000)

## 概要
本資料は、Pythonを用いて文書分類を行うことで、自然言語処理に触れることを目的としています。

## 環境構築
- 形態素解析器MeCabのインストール
- livedoorニュースコーパスの入手

### 形態素解析器のインストール

In [0]:
!apt-get -y install mecab libmecab-dev mecab-ipadic-utf8 > /dev/null
!pip install mecab-python3 > /dev/null
!pip install ngboost > /dev/null

### livedoorニュースコーパスの取得と整理
wgetコマンドによりダウンロードしたものを展開する。

README.txtを表示する。

In [25]:
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz

--2020-04-10 00:03:00--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz.2’

ldcc-20140209.tar.g 100%[===================>]   8.44M  1.89MB/s    in 4.5s    

2020-04-10 00:03:06 (1.89 MB/s) - ‘ldcc-20140209.tar.gz.2’ saved [8855190/8855190]



In [0]:
!tar zxf ldcc-20140209.tar.gz > /dev/null

In [27]:
!cat text/README.txt

livedoor ニュースコーパス


概要
-----------------
本コーパスは、NHN Japan株式会社が運営する「livedoor ニュース」のうち、下記のクリエイティブ・コモンズ
ライセンスが適用されるニュース記事を収集し、可能な限りHTMLタグを取り除いて作成したものです。

- トピックニュース
  http://news.livedoor.com/category/vender/news/
- Sports Watch
  http://news.livedoor.com/category/vender/208/
- ITライフハック
  http://news.livedoor.com/category/vender/223/
- 家電チャンネル
  http://news.livedoor.com/category/vender/kadench/
- MOVIE ENTER
  http://news.livedoor.com/category/vender/movie_enter/
- 独女通信
  http://news.livedoor.com/category/vender/90/
- エスマックス
  http://news.livedoor.com/category/vender/smax/
- livedoor HOMME
  http://news.livedoor.com/category/vender/homme/
- Peachy
  http://news.livedoor.com/category/vender/ldgirls/

収集時期：2012年9月上旬


ライセンス
-----------------
各記事ファイルにはクリエイティブ・コモンズライセンス「表示 - 改変禁止」
（http://creativecommons.org/licenses/by-nd/2.1/jp/）が適用されます。
クレジット表示についてはニュースカテゴリにより異なるため、サブディレクトリにある
それぞれの LICENSE.txt をご覧ください。

livedoor はNHN Japan株式会社の登録商標です。


記事ファイルのフォーマット
-----------------
記事ファイルは以下のフォーマットにしたが

カテゴリとテキストが、どのようなディレクトリやファイルになっているかを確認する。

In [28]:
!ls ./text

CHANGES.txt	kaden-channel	peachy	    sports-watch
dokujo-tsushin	livedoor-homme	README.txt  topic-news
it-life-hack	movie-enter	smax


In [29]:
!ls ./text/sports-watch/ | head

LICENSE.txt
sports-watch-4597641.txt
sports-watch-4601248.txt
sports-watch-4604621.txt
sports-watch-4609913.txt
sports-watch-4612295.txt
sports-watch-4623406.txt
sports-watch-4623588.txt
sports-watch-4627799.txt
sports-watch-4628232.txt


- １行目：記事のURL
- ２行目：記事の日付
- ３行目：記事のタイトル
- ４行目以降：記事の本文

In [30]:
!head ./text/sports-watch/sports-watch-4597641.txt

http://news.livedoor.com/article/detail/4597641/
2010-02-10T10:50:00+0900
【Sports Watch】秋山成勲、メールで吉田に対戦迫った!?
今月8日、都内ホテルでは、総合格闘家・吉田秀彦の引退試合興行「ASTRA」の開催が発表された。

バルセロナ五輪柔道金メダリストとしての実績を引っさげ、2002年にプロ総合格闘家に転向。以後、数々の死闘を繰り広げてきた吉田。昨年大晦日のDynamite!!では、石井慧との金メダリスト対決を制し、4月に迎える引退試合の相手には、桜庭和志やまさかの朝青龍といった報道が駆け巡る中、“反骨の柔道王”秋山成勲が吉田にメールで対戦を迫っていたというのだ。

会見翌日の9日に更新された秋山成勲オフィシャルブログでは、「吉田秀彦対秋山成勲」と題し、「常に憧れ目標にしてきた吉田先輩が引退。正直寂しい気持ちはありますが、ほんまにお疲れ様でした！引退試合はもちろん自分とやるでしょ！？」とストレートに書き綴りながらも、「さっき吉田先輩にメールで自分とやるでしょ？的なメールを打って、軽く流されましたが」と、実際に吉田へ対戦を打診をしていたことを明かした。

もちろん、階級が違う上、秋山は米UFCを主戦場にしている現状、対戦することはまずない。それでも、我が道を行き、空気を一切読まない秋山だけに、案外本気に考えていた可能性もなきにしもあらず——。そんなブログの最後には、「ほんまに柔道から格闘技の道を作って頂いたパイオニアだと自分は思い尊敬してます！引退試合頑張ってください！」と吉田にエールを送る秋山であった。


In [31]:
!ls text/sports-watch | grep ".txt" | wc -l

901


## Pythonで文書分類を実装する

### データの前処理

#### 利用するモジュールのimport
- ファイルを読み込むためのglob
- データ整形を行うためのpandas
- 形態素解析器
- scikit-learn
  - CountVectorizer
  - TFIDFモジュール

In [0]:
import glob
import re

import MeCab
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [0]:
# LabelEncoderでどうしてもWarningが出るので、いったんWarningを非表示にする
import warnings

warnings.simplefilter("ignore")

#### 文書を読み込む
カテゴリは9カテゴリ

記事ファイルのフォーマットは以下であるため、これをタプル化する。

- １行目：記事のURL
- ２行目：記事の日付
- ３行目：記事のタイトル
- ４行目以降：記事の本文

In [0]:
categories = [
    "sports-watch", "topic-news", "dokujo-tsushin", "peachy",
    "movie-enter", "kaden-channel", "livedoor-homme", "smax",
    "it-life-hack",
]

In [0]:
docs = []
for category in categories:
    for f in glob.glob(f"./text/{category}/{category}*.txt"):
        # 1ファイルごとに処理
        with open(f, "r") as fin:
            # nextで1行取得する(__next__を呼ぶ)
            url = next(fin).strip()
            date = next(fin).strip()
            title = next(fin).strip()
            body = "\n".join([line.strip() for line in fin if line.strip()])
      
        docs.append((category, url, date, title, body))

In [36]:
docs[0]

('sports-watch',
 'http://news.livedoor.com/article/detail/5402934/',
 '2011-03-10T13:30:00+0900',
 '【Sports Watch】日本代表・細貝、人気モデルと結婚を発表',
 'アジアカップ・韓国戦では、本田圭佑が外したPKのこぼれ球をゴールに叩き込み、勝利の立役者となったサッカー日本代表・細貝萌。イケメン選手としても知られる細貝は、前橋育英高校卒業後から約6年間所属した浦和レッズを離れ、ドイツのブンデスリーガで新たな挑戦をはじめた。\nそんな折、今月10日、細貝は人生のパートナーとして、人気モデル・中村明花さんとの結婚を自身のブログで発表した。\n「これまでの間、お互いの仕事を支え合いながらお付き合いさせて頂いていました」と綴る細貝。「僕はサッカー選手として自分のリズムで生活を送っている中、明花さんは、時間がある限り美味しい手料理を作ってくれるなど、僕をサポートしてくれました。サポーターの皆様の温かい応援と共に、浦和レッズ在籍時には本当に有り難い存在でした」と明かし、ドイツ挑戦についても「明花さんも一番近くで僕の背中を押してくれました」と語っている。\nまた、中村さんの公式ブログでは「私も日本で仕事があった為、今は別々に暮らしていますが、時期を見て私もドイツに向かおうと思っています」と綴り、ドイツでの新生活をスタートする旨が報告されつつ、「海外に住むなんて考えた事なかったけど…彼を好きになっちゃんたんだからしょうがない！！！笑」とも。\n新たなパートナーを得た細貝、今後の躍進に益々期待が持てそうだ。')

#### 取り扱いやすいデータフレーム形式に変換

In [0]:
df = pd.DataFrame(
        docs,
        columns=["category", "url", "date", "title", "body"],
        dtype="category"
)
# 日付は日付型に変更（今回使うわけでは無い）
df["date"] = pd.to_datetime(df["date"])

#### データの確認

In [38]:
# 先頭行数行を表示
df.head()

,category,url,date,title,body
0,sports-watch,http://news.livedoor.com/article/detail/5402934/,2011-03-10 13:30:00+09:00,【Sports Watch】日本代表・細貝、人気モデルと結婚を発表,アジアカップ・韓国戦では、本田圭佑が外したPKのこぼれ球をゴールに叩き込み、勝利の立役者とな...
1,sports-watch,http://news.livedoor.com/article/detail/6277612/,2012-02-14 11:30:00+09:00,金本知憲が復活へ「何だかんだいったヤツを見返してやりたい」,11日深夜放送、TBS「S1」では「反骨の43歳 金本知憲 復活へ知られざる激闘録」と題し、...
2,sports-watch,http://news.livedoor.com/article/detail/5737499/,2011-07-27 08:30:00+09:00,【Sports Watch】なでしこ岩清水、佐々木監督の暴露に「ふざけんなよ。マジムカつく」,24日放送、テレビ朝日「やべっちF.C.」初の90分スペシャルには、なでしこジャパンから7選...
3,sports-watch,http://news.livedoor.com/article/detail/5184767/,2010-12-04 11:00:00+09:00,【Sports Watch】ヤクルト身売り報道、球団広報は「一切の事実無根」,4日、日刊スポーツは、ヤクルトスワローズがアメーバブログの運営でもお馴染み、サイバーエージェ...
4,sports-watch,http://news.livedoor.com/article/detail/5322376/,2011-02-06 07:30:00+09:00,【Sports Watch】八百長調査委員に、たけし「役に立たねーじゃねーか」,5日、大相撲八百長問題の調査を行う日本相撲協会の特別調査委員会が記者会見を行うと、早稲田大学...


In [39]:
# それぞれの要素数を数え上げる
df.category.value_counts()

sports-watch      900
smax              870
movie-enter       870
it-life-hack      870
dokujo-tsushin    870
kaden-channel     864
peachy            842
topic-news        770
livedoor-homme    511
Name: category, dtype: int64

#### 単語分割
MeCabで分かち書きを行う処理を関数化する。

DataFrameに分かち書き済みのテキストを格納する。

In [0]:
tagger = MeCab.Tagger("-Owakati")

def parse_to_wakati(text):
    # wakatiモードのMeCabのparseは、文字列を返す
    return tagger.parse(text).strip()

In [0]:
# dfのbody_wakatiという列に対して、bodyを分かち書きしたものを代入する
df = df.assign(body_wakati=df.body.apply(parse_to_wakati))

In [42]:
df.head()

,category,url,date,title,body,body_wakati
0,sports-watch,http://news.livedoor.com/article/detail/5402934/,2011-03-10 13:30:00+09:00,【Sports Watch】日本代表・細貝、人気モデルと結婚を発表,アジアカップ・韓国戦では、本田圭佑が外したPKのこぼれ球をゴールに叩き込み、勝利の立役者とな...,アジア カップ ・ 韓国 戦 で は 、 本田 圭 佑 が 外し た PK の こぼれ 球 ...
1,sports-watch,http://news.livedoor.com/article/detail/6277612/,2012-02-14 11:30:00+09:00,金本知憲が復活へ「何だかんだいったヤツを見返してやりたい」,11日深夜放送、TBS「S1」では「反骨の43歳 金本知憲 復活へ知られざる激闘録」と題し、...,11 日 深夜 放送 、 TBS 「 S 1 」 で は 「 反骨 の 43 歳 金本 知 ...
2,sports-watch,http://news.livedoor.com/article/detail/5737499/,2011-07-27 08:30:00+09:00,【Sports Watch】なでしこ岩清水、佐々木監督の暴露に「ふざけんなよ。マジムカつく」,24日放送、テレビ朝日「やべっちF.C.」初の90分スペシャルには、なでしこジャパンから7選...,24 日 放送 、 テレビ朝日 「 や べっち F . C .」 初 の 90 分 スペシャ...
3,sports-watch,http://news.livedoor.com/article/detail/5184767/,2010-12-04 11:00:00+09:00,【Sports Watch】ヤクルト身売り報道、球団広報は「一切の事実無根」,4日、日刊スポーツは、ヤクルトスワローズがアメーバブログの運営でもお馴染み、サイバーエージェ...,4 日 、 日刊 スポーツ は 、 ヤクルトスワローズ が アメーバブログ の 運営 で も...
4,sports-watch,http://news.livedoor.com/article/detail/5322376/,2011-02-06 07:30:00+09:00,【Sports Watch】八百長調査委員に、たけし「役に立たねーじゃねーか」,5日、大相撲八百長問題の調査を行う日本相撲協会の特別調査委員会が記者会見を行うと、早稲田大学...,5 日 、 大相撲 八百長 問題 の 調査 を 行う 日本 相撲 協会 の 特別 調査 委員...


In [43]:
df.body_wakati.head()

0    アジア カップ ・ 韓国 戦 で は 、 本田 圭 佑 が 外し た PK の こぼれ 球 ...
1    11 日 深夜 放送 、 TBS 「 S 1 」 で は 「 反骨 の 43 歳 金本 知 ...
2    24 日 放送 、 テレビ朝日 「 や べっち F . C .」 初 の 90 分 スペシャ...
3    4 日 、 日刊 スポーツ は 、 ヤクルトスワローズ が アメーバブログ の 運営 で も...
4    5 日 、 大相撲 八百長 問題 の 調査 を 行う 日本 相撲 協会 の 特別 調査 委員...
Name: body_wakati, dtype: category
Categories (7359, object): [" ミスター " こと 長嶋 茂雄 ・ 終身 名誉 監督 の 発言 が 物議 を かもし て..., " 合コン 総研 アナリスト " の 水谷 麻衣 さん に 聞く 合コン の 極意 。 有名...,
                            " 女のコ の ため の 日本 最大 級 ファッションフェスタ "『 東京 ガールズコレクシ..., " 女のコ の ため の 日本 最大 級 ファッションフェスタ "『 東京 ガールズコレクシ...,
                            ..., ＮＴＴドコモ が 、 今年 の １２月 を めど に 、 イギリス の ボーダフォン 法人 ...,
                            ＳＦ 小説 の 巨匠 ロバート ・ Ａ ・ ハインライン の 傑作 『 宇宙 の 戦士 』 ..., ＷＢＡ 世界 バンタム 級 王者 ・ 亀田 興 毅 の ４ 度目 の 防衛 戦 が 、 ４月...,
                            ｉＰｈｏｎｅ ５ について 、 様々 な 憶測 が 飛び交っ て いる 。 早く 詳細 を ...]

#### ルールベースのためのラベルのエンコード
scikit-learnでは予測値のラベルは数値にエンコードする必要がある。

まず、ルールを書くために、予測するラベルをエンコードしておく

In [0]:
# ラベルエンコーダは、ラベルを数値に変換する
le = LabelEncoder()

# ラベルをエンコードし、エンコード結果をyに代入する
y = le.fit_transform(df.category)

In [45]:
y[:10]

array([7, 7, 7, 7, 7, 7, 7, 7, 7, 7])

In [46]:
# ラベルエンコーダがエンコードしたクラスの種類を表示
print(le.classes_)

# sports-watchクラスが、何にエンコードされたか確認
print(le.transform(["sports-watch"]))

['dokujo-tsushin' 'it-life-hack' 'kaden-channel' 'livedoor-homme'
 'movie-enter' 'peachy' 'smax' 'sports-watch' 'topic-news']
[7]


#### 学習とテストに分けて、学習と推定を行う

In [0]:
X_train, X_test, y_train, y_test = train_test_split(
                                      df.body_wakati,  # 入力
                                      y, # 正解ラベル
                                      test_size=0.2, # テストデータのサイズ（全体の何割か）
                                      random_state=42, # シャッフルしたときの乱数の種を固定
                                      shuffle=True
)

### ルールベース

In [0]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.metrics import classification_report, confusion_matrix

#### ルールベース分類器の作成

In [0]:
# scikit-learnのBaseEstimatorとTransformerMixinを継承することで、scikit-learn互換のクラスを作りやすくする
class RulebasedEstimator(BaseEstimator, TransformerMixin):
    def __init__(self, label_encoder):
        self.le = label_encoder
  
    def fit(self, X, y):
        return self

    def predict(self, X):
        """ルールを記述"""
        result = []
        for text in X:
            pred = 0
            if re.search(r"(スポーツ|サッカー)", text):
                pred = self.le.transform(["sports-watch"])[0]
            elif re.search(r"(独女|悩み|ブライド|サプライズ)", text):
                pred = self.le.transform(["dokujo-tsushin"])[0]
            elif re.search(r"(Mac|PC|IT|インテル|SSD|CPU)", text):
                pred = self.le.transform(["it-life-hack"])[0]
            elif re.search(r"(電力|冷蔵庫|扇風機|家電|熱中症)", text):
                pred = self.le.transform(["kaden-channel"])[0]
            elif re.search(r"(店|店舗|マラソン|スポーツ)", text):
                pred = self.le.transform(["livedoor-homme"])[0]
            elif re.search(r"(映画|芸能|リアクション|公開)", text):
                pred = self.le.transform(["movie-enter"])[0]
            elif re.search(r"(食べ物|菓子|ごはん|ワイン|カフェ)", text):
                pred = self.le.transform(["peachy"])[0]
            elif re.search(r"(ゲーム|スマホ)", text):
                pred = self.le.transform(["smax"])[0]
            elif re.search(r"(メダル|プレー)", text):
                pred = self.le.transform(["sports-watch"])[0]
            elif re.search(r"(テレビ|ジャーナリスト)", text):
                pred = self.le.transform(["topic-news"])[0]

            result.append(pred)
        return result

#### ルールベースによる分類

In [0]:
rulebased = RulebasedEstimator(label_encoder=le)

In [0]:
rulebased_pred = rulebased.predict(X_test)

####ルールベースの評価

In [52]:
print(classification_report(y_test, rulebased_pred, target_names=le.classes_))

                precision    recall  f1-score   support

dokujo-tsushin       0.19      0.48      0.27       176
  it-life-hack       0.44      0.58      0.50       139
 kaden-channel       0.62      0.19      0.29       178
livedoor-homme       0.15      0.25      0.19        83
   movie-enter       0.54      0.81      0.64       182
        peachy       0.44      0.08      0.14       173
          smax       0.26      0.10      0.15       182
  sports-watch       0.50      0.50      0.50       191
    topic-news       0.23      0.11      0.15       170

      accuracy                           0.35      1474
     macro avg       0.37      0.34      0.31      1474
  weighted avg       0.39      0.35      0.32      1474



  | 実際は正 | 実際は負
--- | -- | --
予測は正 | True Positive | False Positive
予測は負 | False Negative | True Negative

- 適合率(Precision)：正と予測したうち、実際に正であるものの割合

$$ Precision = \frac{True Positive}{True Positive + False Positive} $$

- 再現率(Recall)：実際に正であるもののうち、正と予測されたものの割合

$$ Recall = \frac{True Positive}{True Positive + False Negative} $$

- F値(F-measure)：適合率と再現率の調和平均

$$ F-measure = \frac{2 \cdot Recall \cdot Precision}{Recall + Precision} $$

### 多変数ベルヌーイモデルを用いたナイーブベイズ分類器

データの入力から推定までのパイプラインを作成する。

単語の頻度を数え上げ、それを分類器が学習するというパイプラインを作る。

In [0]:
text_clf = Pipeline([
    ("count_vec", CountVectorizer()),
    ("clf", MultinomialNB()),
])

作成したパイプラインに学習データを流し込む。

In [54]:
text_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('count_vec',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

テストデータを入力し、分類する。

In [0]:
pred = text_clf.predict(X_test)

評価する。

In [56]:
print(classification_report(y_test, pred, target_names=le.classes_))

                precision    recall  f1-score   support

dokujo-tsushin       0.77      0.90      0.83       176
  it-life-hack       0.91      0.91      0.91       139
 kaden-channel       0.95      0.85      0.90       178
livedoor-homme       0.86      0.83      0.85        83
   movie-enter       0.92      0.98      0.95       182
        peachy       0.86      0.74      0.80       173
          smax       0.94      0.99      0.97       182
  sports-watch       0.97      0.96      0.97       191
    topic-news       0.94      0.91      0.92       170

      accuracy                           0.90      1474
     macro avg       0.90      0.90      0.90      1474
  weighted avg       0.91      0.90      0.90      1474



In [57]:
print(confusion_matrix(y_test, pred, labels=le.transform(le.classes_)))

[[158   0   0   1   5  11   0   1   0]
 [  5 126   1   0   0   2   3   1   1]
 [  2   9 152   3   1   2   4   1   4]
 [  2   0   4  69   3   5   0   0   0]
 [  2   0   0   0 179   1   0   0   0]
 [ 30   0   2   5   5 128   3   0   0]
 [  0   1   0   0   0   0 181   0   0]
 [  1   0   0   1   0   0   0 184   5]
 [  6   3   1   1   1   0   1   3 154]]


#### 任意のテキストを入力して解析する

In [58]:
original_text = "あの「Orange is the new black」のシーズン６が7月28日についにNetflixに上陸！"

wakati_text = parse_to_wakati(original_text)
print(le.inverse_transform(text_clf.predict([wakati_text]))[0])

movie-enter


#### 間違った例を見る

In [59]:
idx = y_test != pred
for text, t, p in zip(X_test[idx][:5], y_test[idx][:5], pred[idx][:5]):
    print("予測：{}\n推定:{}".format(le.inverse_transform(t), le.inverse_transform(p)))
    print(text)

ValueError: ignored

### Bernoulliのナイーブベイズ分類器
頻度の数え上げを行い、ナイーブベイズ分類器を学習する。

$$ P(x_{i} | y) = P(i | y)x_{i} + (1-P(i|y))(1-x_{i})$$

In [0]:
bernoulli_clf = Pipeline([
    ("count_vec", CountVectorizer()),
    ("clf", BernoulliNB()),
])

作成したパイプラインに基づき学習する。

In [61]:
bernoulli_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('count_vec',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('clf',
                 BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None,
                             fit_prior=True))],
         verbose=False)

学習したパラメータを用いて、クラス推定を行う。

In [62]:
pred = bernoulli_clf.predict(X_test)
print(classification_report(y_test, pred, target_names=le.classes_))

                precision    recall  f1-score   support

dokujo-tsushin       0.81      0.90      0.85       176
  it-life-hack       0.93      0.89      0.91       139
 kaden-channel       0.83      0.92      0.87       178
livedoor-homme       0.91      0.39      0.54        83
   movie-enter       0.93      0.91      0.92       182
        peachy       0.78      0.73      0.76       173
          smax       0.99      0.98      0.99       182
  sports-watch       0.83      0.99      0.90       191
    topic-news       0.95      0.91      0.93       170

      accuracy                           0.88      1474
     macro avg       0.89      0.85      0.85      1474
  weighted avg       0.88      0.88      0.87      1474



### 線形SVMによる分類器

In [0]:
from sklearn.svm import LinearSVC

In [64]:
svm_clf = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", LinearSVC()),
])

svm_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
               

In [65]:
pred = svm_clf.predict(X_test)
print(classification_report(y_test, pred, target_names=le.classes_))

                precision    recall  f1-score   support

dokujo-tsushin       0.90      0.93      0.91       176
  it-life-hack       0.96      0.98      0.97       139
 kaden-channel       0.97      0.93      0.95       178
livedoor-homme       0.92      0.86      0.89        83
   movie-enter       0.95      0.99      0.97       182
        peachy       0.91      0.88      0.90       173
          smax       0.98      0.99      0.99       182
  sports-watch       0.94      0.98      0.96       191
    topic-news       0.98      0.94      0.96       170

      accuracy                           0.95      1474
     macro avg       0.95      0.94      0.94      1474
  weighted avg       0.95      0.95      0.95      1474



### ランダムフォレストによる分類器

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [67]:
rf_clf = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", RandomForestClassifier()),
])

rf_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None

In [68]:
pred = rf_clf.predict(X_test)
print(classification_report(y_test, pred, target_names=le.classes_))

                precision    recall  f1-score   support

dokujo-tsushin       0.79      0.91      0.85       176
  it-life-hack       0.94      0.94      0.94       139
 kaden-channel       0.91      0.87      0.89       178
livedoor-homme       0.88      0.43      0.58        83
   movie-enter       0.92      0.96      0.94       182
        peachy       0.83      0.81      0.82       173
          smax       0.98      0.99      0.99       182
  sports-watch       0.88      0.97      0.92       191
    topic-news       0.94      0.92      0.93       170

      accuracy                           0.89      1474
     macro avg       0.90      0.87      0.87      1474
  weighted avg       0.90      0.89      0.89      1474



### XGBoostによる分類器
学習に数分時間が必要

In [0]:
import xgboost as xgb

In [88]:
xgb_clf = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", xgb.XGBClassifier()),
])

xgb_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=3,
                        

In [91]:
pred = xgb_clf.predict(X_test)
print(classification_report(y_test, pred, target_names=le.classes_))

                precision    recall  f1-score   support

dokujo-tsushin       0.88      0.90      0.89       176
  it-life-hack       0.95      0.96      0.96       139
 kaden-channel       0.96      0.92      0.94       178
livedoor-homme       0.81      0.76      0.78        83
   movie-enter       0.94      0.96      0.95       182
        peachy       0.83      0.88      0.85       173
          smax       0.99      0.98      0.99       182
  sports-watch       0.92      0.96      0.94       191
    topic-news       0.98      0.91      0.95       170

      accuracy                           0.92      1474
     macro avg       0.92      0.91      0.92      1474
  weighted avg       0.93      0.92      0.92      1474



### NGBoostによる分類器
学習にxgboostのN倍時間が必要
実際には20~30分程度

In [0]:
import ngboost as ngb
from ngboost.distns import k_categorical

In [101]:
#k_categoricalの値は分類ラベル数と同じにする必要がある
ngb_clf = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", ngb.NGBClassifier(Dist=k_categorical(9))),
])

ngb_clf.fit(X_train, y_train)

[iter 0] loss=2.1875 val_loss=0.0000 scale=0.5000 norm=5.4142
[iter 100] loss=0.8331 val_loss=0.0000 scale=2.0000 norm=7.0513
[iter 200] loss=0.5230 val_loss=0.0000 scale=4.0000 norm=10.8217
[iter 300] loss=0.4314 val_loss=0.0000 scale=1.0000 norm=2.4859
[iter 400] loss=0.4042 val_loss=0.0000 scale=1.0000 norm=2.4305


Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                                                          presort='deprecated',
                                                          random_state=None,
                                                          splitter='best'),
                               Dist=<class 'ngboost.distns.categorica

In [0]:
#学習モデル保存
import pickle

In [125]:
pickle.dump(ngb_clf, open("model_ngb_best.pkl", "wb"))

AttributeError: ignored

In [108]:
#分類結果推定
pred = ngb_clf.predict(X_test)
print(classification_report(y_test, pred, target_names=le.classes_))

                precision    recall  f1-score   support

dokujo-tsushin       0.88      0.69      0.77       176
  it-life-hack       0.74      0.88      0.80       139
 kaden-channel       0.87      0.80      0.83       178
livedoor-homme       0.78      0.52      0.62        83
   movie-enter       0.87      0.92      0.90       182
        peachy       0.57      0.84      0.68       173
          smax       0.99      0.98      0.99       182
  sports-watch       0.92      0.84      0.88       191
    topic-news       0.95      0.84      0.89       170

      accuracy                           0.83      1474
     macro avg       0.84      0.81      0.82      1474
  weighted avg       0.85      0.83      0.83      1474



### 多層パーセプトロンによる分類器

学習に数分時間が必要

In [0]:
from sklearn.neural_network import MLPClassifier

In [70]:
mlp_clf = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MLPClassifier()),
])

mlp_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                               batch_size='auto', beta_1=0.9, beta_2=0.999,
                               early_stopping=False, epsilon=1e-08,
                               hidden_layer_sizes=(100,),
                               learning_rate='constant',
                               learning_rat

In [71]:
pred = mlp_clf.predict(X_test)
print(classification_report(y_test, pred, target_names=le.classes_))

                precision    recall  f1-score   support

dokujo-tsushin       0.90      0.92      0.91       176
  it-life-hack       0.96      0.96      0.96       139
 kaden-channel       0.98      0.93      0.95       178
livedoor-homme       0.94      0.87      0.90        83
   movie-enter       0.94      0.98      0.96       182
        peachy       0.90      0.88      0.89       173
          smax       0.97      0.99      0.98       182
  sports-watch       0.95      0.98      0.97       191
    topic-news       0.97      0.94      0.96       170

      accuracy                           0.94      1474
     macro avg       0.94      0.94      0.94      1474
  weighted avg       0.94      0.94      0.94      1474



### 多変数ベルヌーイモデルをnumpyで実装
まだfor文があり、不完全な実装で動作が遅いです。

In [0]:
import numpy as np

from scipy.sparse import lil_matrix, csr_matrix


class NaiveBayes(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.vocabularies = None
        self.categories = None
        self.category_count = None
        self.category_word_occurence = None
        self.denominator = {}
    
    def fit(self, X, y):
        self.vocabularies = np.unique(X)
        self.categories, self.category_count = np.unique(y, return_counts=True)
        
        self.category_word_occurrence = lil_matrix((self.categories.shape[0], X.shape[1]))
        for c in self.categories:
            _y = y[y == c]
            _X = X[y == c]
            self.category_word_occurrence[c] = np.sum(_X, axis=0)
        self.category_word_occurrence = self.category_word_occurrence.tocsr()
        
        # P(word | cat)の分母 Σ |(cat, word)| + |V|
        for cat in self.categories:
            self.denominator[cat] = np.sum(self.category_word_occurrence[cat]) + len(self.vocabularies)
    
    def word_prob(self, word, category):
        """P(word|cat)"""
        return (self.category_word_occurrence[category, word]+1) / self.denominator[category]
    
    def score(self, doc, cat):
        """P(doc|cat)"""
        sc = np.log(self.category_count[cat] / self.category_count.sum())
        for word in doc.nonzero()[1]:
            sc += np.log(self.word_prob(word, cat))
        return sc
    
    def predict(self, X):
        return np.array([self._predict(doc) for doc in X])
    
    def _predict(self, doc):
        """P(cat|doc)"""
        return np.argmax(np.array([self.score(doc, cat) for cat in self.categories]))

In [0]:
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)

In [0]:
nb_clf = NaiveBayes()
nb_clf.fit(X_train_vec, y_train)

In [75]:
pred = nb_clf.predict(vectorizer.transform(X_test))
print(classification_report(y_test, pred, target_names=le.classes_))

                precision    recall  f1-score   support

dokujo-tsushin       0.86      0.89      0.87       176
  it-life-hack       0.94      0.86      0.90       139
 kaden-channel       0.93      0.91      0.92       178
livedoor-homme       0.73      0.83      0.78        83
   movie-enter       0.93      0.97      0.95       182
        peachy       0.85      0.74      0.79       173
          smax       0.97      0.95      0.96       182
  sports-watch       0.94      0.94      0.94       191
    topic-news       0.89      0.96      0.92       170

      accuracy                           0.90      1474
     macro avg       0.89      0.90      0.89      1474
  weighted avg       0.90      0.90      0.90      1474



## ルールベースの改善をしてみよう

In [76]:
df.query("category == 'smax'").head()

,category,url,date,title,body,body_wakati
5627,smax,http://news.livedoor.com/article/detail/6814359/,2012-08-01 20:55:00+09:00,本日発売開始！NTTドコモ向け簡単操作のシニア層向けスマホ「らくらくスマートフォン F-12...,らくらくスマートフォン F-12Dの画面キャプチャの方法を紹介！\n本日1日（水）にNTTド...,らくらく スマート フォン F - 12 D の 画面 キャプチャ の 方法 を 紹介 ！ ...
5628,smax,http://news.livedoor.com/article/detail/6826557/,2012-08-05 21:55:00+09:00,【最近のオススメ「Androidアプリ」特集：2012年7月30日〜8月5日編】,2012年7月30日〜8月5日に紹介したAndroidアプリ！\n先週はNTTドコモ向け「ら...,2012 年 7 月 30 日 〜 8 月 5 日 に 紹介 し た Android アプリ...
5629,smax,http://news.livedoor.com/article/detail/6578513/,2012-05-21 09:55:00+09:00,モバイルプロジェクト・アワード2012の審査結果が発表！GALAXY Noteがハードウェア...,モバイルハードウェア部門最優秀賞をGALAXY Noteが受賞！\nモバイル・コンテンツ・フ...,モバイル ハードウェア 部門 最優秀 賞 を GALAXY Note が 受賞 ！ モバイル...
5630,smax,http://news.livedoor.com/article/detail/6872644/,2012-08-21 09:55:00+09:00,iOSにもガラパゴスがやってきた！Appleにはない電子書籍の自由化「電子書籍 GALAPA...,iPhone、iPadでもGALAPAGOS！\n昨年、シャープ製電子書籍端末やスマートフォ...,iPhone 、 iPad で も GALAPAGOS ！ 昨年 、 シャープ 製 電子 書...
5631,smax,http://news.livedoor.com/article/detail/6601757/,2012-05-28 13:55:00+09:00,auスマートフォン「HTC J ISW13HT」のICカードロックがない！USIMカードを差...,レベル2ロックがかかってない！\nau向けに先週25日（金）に発売開始したAndroidスマ...,レベル 2 ロック が かかっ て ない ！ au 向け に 先週 25 日 （ 金 ） に...


In [0]:
# scikit-learnのBaseEstimatorとTransformerMixinを継承することで、scikit-learn互換のクラスを作りやすくする
class RulebasedEstimator(BaseEstimator, TransformerMixin):
    def __init__(self, label_encoder):
        self.le = label_encoder
  
    def fit(self, X, y):
        return self

    def predict(self, X):
        """ルールを記述"""
        result = []
        for text in X:
            pred = 0
            if re.search(r"(スポーツ|サッカー)", text):
                pred = self.le.transform(["sports-watch"])[0]
            elif re.search(r"(独女|悩み|ブライド|サプライズ)", text):
                pred = self.le.transform(["dokujo-tsushin"])[0]
            elif re.search(r"(Mac|PC|IT|インテル|SSD|CPU)", text):
                pred = self.le.transform(["it-life-hack"])[0]
            elif re.search(r"(電力|冷蔵庫|扇風機|家電|熱中症)", text):
                pred = self.le.transform(["kaden-channel"])[0]
            elif re.search(r"(店|店舗|マラソン|スポーツ)", text):
                pred = self.le.transform(["livedoor-homme"])[0]
            elif re.search(r"(映画|芸能|リアクション|公開)", text):
                pred = self.le.transform(["movie-enter"])[0]
            elif re.search(r"(食べ物|菓子|ごはん|ワイン|カフェ)", text):
                pred = self.le.transform(["peachy"])[0]
            elif re.search(r"(ゲーム|スマホ)", text):
                pred = self.le.transform(["smax"])[0]
            elif re.search(r"(メダル|プレー)", text):
                pred = self.le.transform(["sports-watch"])[0]
            elif re.search(r"(テレビ|ジャーナリスト)", text):
                pred = self.le.transform(["topic-news"])[0]

            result.append(pred)
        return result

In [78]:
rulebased = RulebasedEstimator(label_encoder=le)
rulebased_pred = rulebased.predict(X_test)
print(classification_report(y_test, rulebased_pred, target_names=le.classes_))

                precision    recall  f1-score   support

dokujo-tsushin       0.19      0.48      0.27       176
  it-life-hack       0.44      0.58      0.50       139
 kaden-channel       0.62      0.19      0.29       178
livedoor-homme       0.15      0.25      0.19        83
   movie-enter       0.54      0.81      0.64       182
        peachy       0.44      0.08      0.14       173
          smax       0.26      0.10      0.15       182
  sports-watch       0.50      0.50      0.50       191
    topic-news       0.23      0.11      0.15       170

      accuracy                           0.35      1474
     macro avg       0.37      0.34      0.31      1474
  weighted avg       0.39      0.35      0.32      1474

